### Set som Globals

In [1]:
PATH_RELATIVE = 'data/'
LABEL = 'label_users_top_100'
LABEL = 'label_bins'
LABEL = 'label_time_encoded'
LABEL_MODEL = 'Responsible'
LABEL_MODEL = 'Time-Bins'
LABEL_MODEL = 'Time-Encoded'

### 1. Loading in the model and tokenizer
The `from_pretrained` parameter can simply be changed to the saved output folder if you wan't to
continue training an already fine-tuned model.

In [3]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np

model = TFAutoModelForSequenceClassification.from_pretrained(f'{PATH_RELATIVE}models/IHLP-XLM-RoBERTa-{LABEL_MODEL}')
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at data/models/IHLP-XLM-RoBERTa-Time-Encoded.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [4]:
df = pd.read_csv(f'{PATH_RELATIVE}cached_test_{LABEL}.csv')
df = df[-1000:]

print(df.head())

                                                    text  label
50534  oprettelse af ny ansat: pmmoeller (start: --) ...    483
50535  side via biblioteket melder om fejl (ddos prot...    117
50536  problem hello, i have a problem. on fridayi ha...    473
50537                                  bestilling kabler    133
50538  luk/nedlg sdu-ansat: nicoline lyck bech, nbech...    482


In [5]:
from collections import defaultdict

def tokenize_texts(sentences, max_length=512, padding='max_length'):
    return tokenizer(
        sentences,
        truncation=False,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

tokenized_text = dict(tokenize_texts(list(df['text'].values)))

In [56]:
import tensorflow as tf

model.compile(metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10)])
results = model.evaluate(tokenized_text, df.label.values, batch_size=16, verbose=False)

print(results)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


[3.838249921798706, 0.5440000295639038]
